In [ ]:
from ipymlwidgets.widgets import Box, ImageAnnotated, Text
from ipymlwidgets.utils import demo_image

el = Box([Text("Hello"), ImageAnnotated(demo_image())], layout={"border-radius" : "10px", "width" : "50%", "margin": "10px"})
display(el)

el.children[1].observe(lambda x: el.children[0].set_text(x["new"]), names="selection")


In [ ]:
from ipymlwidgets.widgets import Canvas

canvas = Canvas()
display(canvas)

canvas.size = (50,50)
canvas.fill_color = "red"
canvas.draw_rect((0,0,50,50))

MultiCanvas((8, 8), layers=1)

In [ ]:
import ipywidgets as W
from ipymlwidgets.traits import SupportedTensor
from ipymlwidgets.utils import demo_image
from ipymlwidgets.widgets.ocr import ItemOCR
from ipymlwidgets.widgets import Box


box = Box()
box.add_child(ItemOCR(demo_image(), "1"))
box.add_child(ItemOCR(demo_image(), "2"))
box.layout = {"display": "flex", "flex-direction": "column", "gap": "20px"}

display(box)


In [1]:
import ipywidgets as W
from ipymlwidgets.traits import SupportedTensor
from ipymlwidgets.demo import demo_image
from ipymlwidgets.widgets.ocr import ItemOCR
from ipymlwidgets.widgets import Box, ImageAnnotated
from typing import Optional
import numpy as np

class OCRAnnotated(Box):
    """A widget that supports OCR annotation in an image."""

    def __init__(self, 
        image : Optional[SupportedTensor] = None, 
        boxes : Optional[SupportedTensor] = None
    ):
        self._image = ImageAnnotated(image)
        self._div_image = Box([self._image], layout={"gap": "10px", "width": "50%"})
        self._div_items = Box([], layout={"display": "flex", "flex-direction": "column", "gap": "10px", "width": "50%"})

        super().__init__([self._div_image, self._div_items], layout={"display": "flex", "flex-direction": "row", "margin" : "10px", "gap": "20px"})

        self._image.observe(self._on_boxes_change, names="boxes")
        self._image.observe(self._on_selection_change, names="selection")
        self._div_items_order = [] # list of indices of the items in the order they are displayed

        #self._image.observe(self._on_mouse_up, names="mouse_up")
    def _on_boxes_change(self, change):
        old_len = len(change['old']) if change['old'] is not None else 0
        new_len = len(change['new'])
        if old_len < new_len: # a new box was added

            # TODO handle the case where boxes are assigned directly? hmm..
            new_box = change['new'][-1]
            self._div_items_order.insert(0, len(self._div_items.children))
            cropped = self._image.crop([-1])[0]
            item = ItemOCR(cropped, f"{len(self._div_items.children)} {new_box}")
            item.observe(self._on_item_close, names="click_close")
            self._div_items.insert_child(item, 0)
            text.value = f"{len(self._div_items.children)} {self._div_items_order}"

        elif old_len > new_len: # a box was deleted
            pass # TODO
        else:
            # a box was updated
            diff = np.any(change['new'] != change['old'], axis=1)
            index = np.nonzero(diff)[0].tolist()
            crops = self._image.crop(index)
            for i, crop in zip(index, crops):
                j = self._resolve_index(i)
                item = self._div_items.children[j]
                item.set_image(crop)
                #self._div_items.replace_child(ItemOCR(crop, f"{len(self._div_items.children)} {change['new'][i]}"), j)

            #self._div_items.send_state()

          

            
            #text.value = f"{len(self._div_items.children)} {change['new']}  {change['old']} {index}"
    
    def _on_item_close(self, change):
        item = change['owner']
        # delete the child
        j = self._div_items.index_child(item)
        i = self._div_items_order[j] # get the box index
        self._div_items.remove_child(item)
        # delete the box
        self._image.remove_box(i)

    def _on_selection_change(self, change):
        new, old = change['new'], change['old']
        if new is None:
            return # box was deselected
        if new.index < 0:
            return # box is being created - it hasnt been added yet
        prev = list(self._div_items_order)
        i, j = 0, self._resolve_index(new.index)
        self._swap_children(i, j)
        text.value = f"{len(self._div_items.children)} {new.index} {prev} {self._div_items_order}, {self._div_items.children}"

    def _resolve_index(self, i):
        # TODO this seems dumb, better would be self._div_items_order[i]!
        return self._div_items_order.index(i)

    def _swap_children(self, i, j):        
        _i, _j = self._div_items_order[i], self._div_items_order[j]
        self._div_items_order[j], self._div_items_order[i] = _i, _j
        c_i = self._div_items.children[i]
        c_j = self._div_items.children[j]
        self._div_items.children[i] = c_j
        self._div_items.children[j] = c_i
        self._div_items.send_state()
        
ocr = OCRAnnotated(demo_image())
display(ocr)

text = W.Text(value="", layout={"width": "100%", "height": "100%", "border": "none", "padding": "0", "margin": "0"})
display(text)



OCRAnnotated(children=[Box(children=[MultiCanvas((32, 32), layers=3)], layout={'gap': '10px', 'width': '50%'})…

Text(value='', layout=Layout(border_bottom='none', border_left='none', border_right='none', border_top='none',…

(0, 0)